<h1>Try to get Iceland's mobility data</h1>

In [3]:
import requests
import pandas as pd
from matplotlib import pyplot as plt
from scipy.stats import gamma
import numpy as np

<h2>Helper methods for accessing the API</h2>

The helper methods in c3aidatalake.py convert a JSON response from the C3.ai APIs to a Pandas DataFrame.

In [4]:
import c3aidatalake as c3

<h1>Pobranie czegokolwiek</h1>
Filtrując zwróconą tabelę po wartościach w kolumnach<br>
Wynik to pandas.DataFrame

In [3]:
location =  c3.fetch(
    "outbreaklocation",{"spec": {"filter" : "name != 'NaN' && locationType == 'country' && id == 'Poland'"
        }
    }
)
location

,latestTotalPopulation,populationOfAllChildren,locationType,populationCDS,countryArea,countryCode,population2018,population2019,id,name,typeIdent,hospitalPrediction.timestamp,location.value.id,location.timestamp,fips.id
0,32738308.0,32738308.0,country,38433600,304255.0,PL,37978548,37972812,Poland,Poland,EP_LOC,2019-01-01T00:00:00,Poland,2020-08-14T00:00:00Z,PL


Lepiej używać countryCode według <a href="https://en.wikipedia.org/wiki/List_of_FIPS_country_codes">kodów krajów FIPS</a><br> To jest ustandaryzowane

In [4]:
location =  c3.fetch(
    "outbreaklocation",{"spec": {"filter" : "name != 'NaN' && locationType == 'country' && ( countryCode == 'PL' || countryCode == 'IC')"
        }
    }
)
location

,latestTotalPopulation,populationOfAllChildren,locationType,populationCDS,countryArea,countryCode,population2018,population2019,id,name,typeIdent,hospitalPrediction.timestamp,location.value.id,location.timestamp,fips.id
0,406766.0,406766.0,country,364260,100250.0,IC,353574,356991,Iceland,Iceland,EP_LOC,2019-01-01T00:00:00,Iceland,2020-08-14T00:00:00Z,IS
1,32738308.0,32738308.0,country,38433600,304255.0,PL,37978548,37972812,Poland,Poland,EP_LOC,2019-01-01T00:00:00,Poland,2020-08-14T00:00:00Z,PL


<h1>Ściągnięcie wszystkich rekordów</h1>
Zakładając domyślny limit pobieranych rekordów wg: <a href="https://c3.ai/covid-19-api-documentation/#section/Using-C3.ai-APIs/Limits">dokumentacji</a>

In [20]:
off = 0
while True: 
    respall = c3.fetch("diagnosis",
        {
            "spec" : {
                "filter": "contains(testResults, 'COVID-19')",
                "include" : "this, diagnostics.source, diagnostics.key, diagnostics.value",
                "offset" : off
            }
        }
    )
    off += 2000
    if respall.shape[0] != 2000: break
respall

,diagnostics,imageUrl,source,idPatient,age,testResults,clinicalNotes,id,location.id,temperature
0,[{'parent': {'id': 'covid-19-pneumonia-30-L.jp...,MONTREAL/images/covid-19-pneumonia-30-L.jpg,Montreal,93,55.0,COVID-19: positive,Fever and non-productive cough start from 5 da...,covid-19-pneumonia-30-L.jpg,Tehran_Iran,NaN
1,[{'parent': {'id': 'covid-19-pneumonia-30-PA.j...,MONTREAL/images/covid-19-pneumonia-30-PA.jpg,Montreal,93,55.0,COVID-19: positive,Fever and non-productive cough start from 5 da...,covid-19-pneumonia-30-PA.jpg,Tehran_Iran,NaN
2,[{'parent': {'id': 'covid-19-pneumonia-34.png'...,MONTREAL/images/covid-19-pneumonia-34.png,Montreal,165,NaN,COVID-19: positive,Patient admitted for heart failure. Developed ...,covid-19-pneumonia-34.png,Rome_Italy,NaN
3,[{'parent': {'id': 'covid-19-pneumonia-35-1.jp...,MONTREAL/images/covid-19-pneumonia-35-1.jpg,Montreal,167,75.0,COVID-19: positive,"Fever for ten days with dry cough, shortness o...",covid-19-pneumonia-35-1.jpg,Riccione_Italy,NaN
4,[{'parent': {'id': 'covid-19-pneumonia-35-2.jp...,MONTREAL/images/covid-19-pneumonia-35-2.jpg,Montreal,167,75.0,COVID-19: positive,"Fever for ten days with dry cough, shortness o...",covid-19-pneumonia-35-2.jpg,Riccione_Italy,NaN
...,...,...,...,...,...,...,...,...,...,...
153,"[{'parent': {'id': 'tpmd200203f2-b.png'}, 'key...",MONTREAL/images/tpmd200203f2-b.png,Montreal,332,56.0,COVID-19: positive,NaN,tpmd200203f2-b.png,Thailand,36.7
154,"[{'parent': {'id': 'tpmd200203f2-c.png'}, 'key...",MONTREAL/images/tpmd200203f2-c.png,Montreal,332,56.0,COVID-19: positive,NaN,tpmd200203f2-c.png,Thailand,36.8
155,[{'parent': {'id': 'yxppt-2020-02-19_00-51-27_...,MONTREAL/images/yxppt-2020-02-19_00-51-27_2872...,Montreal,187,50.0,COVID-19: positive,50-year-old man was sent to the fever clinic f...,yxppt-2020-02-19_00-51-27_287214-day10.jpg,China,NaN
156,[{'parent': {'id': 'yxppt-2020-02-19_00-51-27_...,MONTREAL/images/yxppt-2020-02-19_00-51-27_2872...,Montreal,187,50.0,COVID-19: positive,50-year-old man was sent to the fever clinic f...,yxppt-2020-02-19_00-51-27_287214-day12.jpg,China,NaN


In [39]:
respall = c3.fetch("diagnosisdetail",
    {
        "spec" : {
            "limit" : "10"
        }
    }
)
respall#respall[["age"]]

,key,value,source,id,parent.id
0,RT_PCR_positive,Y,Montreal,RT_PCR_positive_000001-9.jpg,000001-9.jpg
1,RT_PCR_positive,Unclear,Montreal,RT_PCR_positive_00870a9c.jpg,00870a9c.jpg
2,RT_PCR_positive,Y,Montreal,RT_PCR_positive_01E392EE-69F9-4E33-BFCE-E5C968...,01E392EE-69F9-4E33-BFCE-E5C968654078.jpeg
3,RT_PCR_positive,Y,Montreal,RT_PCR_positive_03BF7561-A9BA-4C3C-B8A0-D3E585...,03BF7561-A9BA-4C3C-B8A0-D3E585F73F3C.jpeg
4,RT_PCR_positive,Y,Montreal,RT_PCR_positive_071d06607edf81d70c940e043bce34...,071d06607edf81d70c940e043bce34_jumbo.jpeg
5,RT_PCR_positive,Unclear,Montreal,RT_PCR_positive_08d780ae.jpg,08d780ae.jpg
6,RT_PCR_positive,Y,Montreal,RT_PCR_positive_08ed451c576ee00935ee178ec85c57...,08ed451c576ee00935ee178ec85c57_jumbo-1.jpeg
7,RT_PCR_positive,Y,Montreal,RT_PCR_positive_0a6c60063b4bae4de001caaba306d1...,0a6c60063b4bae4de001caaba306d1_jumbo.jpeg
8,RT_PCR_positive,Unclear,Montreal,RT_PCR_positive_0a7faa2a.jpg,0a7faa2a.jpg
9,RT_PCR_positive,Y,Montreal,RT_PCR_positive_1-s2.0-S0085253820303616-gr1_l...,1-s2.0-S0085253820303616-gr1_lrg-a.png
